In [1]:
from DatabaseHandler.MongoDBHandler import MongoDBHandler
from DatabaseHandler.PostgresHandler import PostgresHandler
from pandas import DataFrame as PandasDataFrame


postgresdb = PostgresHandler()
mongodb = MongoDBHandler()

#connect to todoist database
mongodb.connect("todoist")
postgresdb.connect("todoist")

#load data from mongodb
comments = mongodb.read_data("comments", {})
opened_tasks = mongodb.read_data("opened_tasks", {})
completed_tasks = mongodb.read_data("completed_tasks", {})
projects = mongodb.read_data("projects", {})    
productivity_stats = mongodb.read_data("productivity_stats", {})

In [56]:
raw_comments_df = PandasDataFrame(comments)
raw_comments_df = raw_comments_df.drop_duplicates(subset='id', keep="first")
comments_df = raw_comments_df[['id', 'content','task_id', 'posted_at',  'extraction_datetime']]

attchement_none_row = {
    'resource_type': None,
    'file_name': None,
    'file_size': None,
    'file_type': None,
    'file_url': None,
    'file_duration': None,
    'upload_state': None,
    'image': None,
    'image_width': None,
    'image_height': None,
    'url': None,
    'title': None
  }
attachments = [el if el != None else attchement_none_row  for el in raw_comments_df['attachment'].to_list() ]
attachments_df = PandasDataFrame(attachments)
attachments_df['id'] = raw_comments_df['_id'].astype(str)
comments_df['attachment_id'] = attachments_df['id']
attachments_df

/tmp/ipykernel_14180/4218246713.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comments_df['attachment_id'] = attachments_df['id']


,resource_type,file_name,file_size,file_type,file_url,file_duration,upload_state,image,image_width,image_height,url,title,id
0,image,image.png,76194.0,image/png,https://files.todoist.com/hhTqMv04xzQzmrxc6-lf...,None,completed,https://files.todoist.com/hhTqMv04xzQzmrxc6-lf...,1259.0,530.0,None,None,658ed1523dea970d4ebb411c
1,image,image.png,3329.0,image/png,https://files.todoist.com/9wp6WWCh0GutaqWJocR8...,None,completed,https://files.todoist.com/9wp6WWCh0GutaqWJocR8...,460.0,72.0,None,None,658ed1523dea970d4ebb411d
2,file,Attention Is All You Need - Google.pdf,2215244.0,application/pdf,https://files.todoist.com/QoWDWx759PqJwUXaUUOh...,None,completed,None,NaN,NaN,None,None,658ed1523dea970d4ebb411e
3,file,Zero-shot Text-to-SQL with ChatGPT.pdf,2053749.0,application/pdf,https://files.todoist.com/Nf4yB3QfumBSudbaIYX4...,None,completed,None,NaN,NaN,None,None,658ed1523dea970d4ebb411f
4,file,DIN-SQL_Decomposed In-Context Learning of.pdf,1457344.0,application/pdf,https://files.todoist.com/zcidiGaMP-o2MrpsQXLT...,None,completed,None,NaN,NaN,None,None,658ed1523dea970d4ebb4120
5,image,image.png,152315.0,image/png,https://files.todoist.com/qPk4gl7oLlKN2bCkD5Qm...,None,completed,https://files.todoist.com/qPk4gl7oLlKN2bCkD5Qm...,1441.0,500.0,None,None,658ed1523dea970d4ebb4121
6,image,image.png,103803.0,image/png,https://files.todoist.com/5ARrqMjKiqyMYbWtXkbt...,None,completed,https://files.todoist.com/5ARrqMjKiqyMYbWtXkbt...,902.0,402.0,None,None,658ed1523dea970d4ebb4122
7,image,image.png,41940.0,image/png,https://files.todoist.com/Oe5pvt73vZs7p3QzK40C...,None,completed,https://files.todoist.com/Oe5pvt73vZs7p3QzK40C...,1360.0,599.0,None,None,658ed1523dea970d4ebb4123
8,image,image.png,34624.0,image/png,https://files.todoist.com/I5P1nKsrwL1UAd9WQJBF...,None,completed,https://files.todoist.com/I5P1nKsrwL1UAd9WQJBF...,282.0,636.0,None,None,658ed1523dea970d4ebb4124
9,None,None,NaN,None,None,None,None,None,NaN,NaN,None,None,658ed1523dea970d4ebb4125


In [55]:
comments_df = comments_df.drop_duplicates(subset='id', keep='first')
comments_df[comments_df['id'] == '3321888008']

,id,content,task_id,posted_at,extraction_datetime,attachment_id
0,3321888008,"Ta esquisito, verificar o porque",7024838790,2023-07-18 13:43:26.618,2023-12-25 06:48:10,658ed1523dea970d4ebb411c


In [26]:


from sqlalchemy import create_engine
from Utils.file_handler import read_yaml
from Env.paths import CONFIG_PATH


spark = SparkSession.builder.getOrCreate()
pd_df = PandasDataFrame(comments)
schema = infer_schema(pd_df)
configs = read_yaml(CONFIG_PATH + 'app_config.yaml')
postgres_configs = configs['databases']['postgres']
host = postgres_configs['host']
port = postgres_configs['port']
user = postgres_configs['user']
password = postgres_configs['password']
driver = postgres_configs['driver']
db_name='todoist'


conn_str = f'postgresql://{user}:{password}@{host}:{port}/{db_name}'
postgresdb = create_engine(conn_str)

pd_df = pd_df.drop(columns=['attachment'])
pd_df['_id'] = pd_df['_id'].astype(str)

pd_df.to_sql('comments', postgresdb.engine, if_exists='replace', index=False)

10

In [101]:
raw_prod_stats_df = PandasDataFrame(productivity_stats)
prod_stats_df =raw_prod_stats_df[['date','total_completed','extraction_datetime']]
prod_stats_df['item_id'] = raw_prod_stats_df['_id'].astype(str)
#postgresdb.write_data('productivitystats', prod_stats_df)

#load items

items_set = raw_prod_stats_df[['date','items']].to_dict('records')
items  = [{'date': items['date'], **item} for items in items_set for item in items['items']]
items_df= PandasDataFrame(items)
items_df = prod_stats_df[['date','item_id']].merge(items_df, on='date')
items_df.columns = ['date', 'id','completed','project_id']
#postgresdb.write_data('items', items_df)


/tmp/ipykernel_14180/4043275482.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prod_stats_df['item_id'] = raw_prod_stats_df['_id'].astype(str)


In [122]:
raw_prod_stats_df = PandasDataFrame(productivity_stats)
prod_stats_df =raw_prod_stats_df[['date','total_completed','extraction_datetime','_id']]
prod_stats_df['id'] = raw_prod_stats_df['_id'].astype(str)




items_set = raw_prod_stats_df[['date','items']].to_dict('records')
items  = [{'date': items['date'], **item} for items in items_set for item in items['items']]
items_df= PandasDataFrame(items)
items_df = prod_stats_df[['date','id']].merge(items_df, on='date').drop(['completed'], axis=1)
items_df.columns = ['date', 'prod_stats_id','project_id']
items_df

,date,prod_stats_id,project_id
0,2023-12-25,658ed1533dea970d4ebb41f8,2298360468
1,2023-12-25,658ed1533dea970d4ebb41f8,2299428438
2,2023-12-25,658ed1533dea970d4ebb41f8,2316085252
3,2023-12-25,658ed1533dea970d4ebb41f8,2298360468
4,2023-12-25,658ed1533dea970d4ebb41f8,2299428438
...,...,...,...
363,2023-12-26,659077b00539bb4ddf0f9c07,2299428438
364,2023-12-26,659077b00539bb4ddf0f9c07,2298360468
365,2023-12-26,659077b00539bb4ddf0f9c07,2299428438
366,2023-12-26,659077b00539bb4ddf0f9c07,2298360468


In [21]:
from pandas import concat

raw_opened_tasks_df = PandasDataFrame(opened_tasks)
raw_completed_tasks_df = PandasDataFrame(completed_tasks) 

tasks_cols=['_id','project_id','section_id','content','extraction_datetime']
tasks_df =  concat([raw_opened_tasks_df[tasks_cols], raw_completed_tasks_df[tasks_cols]], axis=0)
tasks_df.columns = ['id','project_id','section_id','content','extraction_datetime']
tasks_df['id'] = tasks_df['id'].astype(str)

completed_tasks_df = raw_completed_tasks_df[['_id','completed_at','user_id','extraction_datetime']]
completed_tasks_df.columns = ['id','completed_at','user_id','extraction_datetime']
completed_tasks_df['id'] = completed_tasks_df['id'].astype(str)

opened_tasks_df = raw_opened_tasks_df[[
                        '_id','assignee_id','comment_count','created_at',
                        'creator_id','description','due','labels','order',
                        'parent_id','priority','url','extraction_datetime'
                        ]]

opened_tasks_df.columns =[
                        'id','assignee_id','comment_count','created_at',
                        'creator_id','description','due','labels','order',
                        'parent_id','priority','url','extraction_datetime']

opened_tasks_df['id'] = opened_tasks_df['id'].astype(str)

/tmp/ipykernel_4261/1864667355.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  completed_tasks_df['id'] = completed_tasks_df['id'].astype(str)
/tmp/ipykernel_4261/1864667355.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opened_tasks_df['id'] = opened_tasks_df['id'].astype(str)


In [30]:
comments_df[['attachment_id','extraction_datetime']]

,attachment_id,extraction_datetime
0,658ed1523dea970d4ebb411c,2023-12-25 06:48:10
1,658ed1523dea970d4ebb411d,2023-12-25 06:48:10
2,658ed1523dea970d4ebb411e,2023-12-25 06:48:10
3,658ed1523dea970d4ebb411f,2023-12-25 06:48:10
4,658ed1523dea970d4ebb4120,2023-12-25 06:48:10
5,658ed1523dea970d4ebb4121,2023-12-25 06:48:10
6,658ed1523dea970d4ebb4122,2023-12-25 06:48:10
7,658ed1523dea970d4ebb4123,2023-12-25 06:48:10
8,658ed1523dea970d4ebb4124,2023-12-25 06:48:10
9,658ed1523dea970d4ebb4125,2023-12-25 06:48:10


In [35]:
attachments_df.columns = ['resource_type', 'file_name', 'file_size', 'file_type', 'file_url',
       'file_duration', 'upload_state', 'image', 'image_width', 'image_height',
       'url', 'title', 'id']

Index(['resource_type', 'file_name', 'file_size', 'file_type', 'file_url',
       'file_duration', 'upload_state', 'image', 'image_width', 'image_height',
       'url', 'title', 'attachment_id'],
      dtype='object')

In [47]:
raw_opened_tasks_df = PandasDataFrame(opened_tasks)
raw_completed_tasks_df = PandasDataFrame(completed_tasks) 

tasks_cols=['id','project_id','section_id','content','extraction_datetime']
tasks_df =  concat([raw_opened_tasks_df[tasks_cols], raw_completed_tasks_df[tasks_cols]], axis=0)
tasks_df.columns = ['id','project_id','section_id','content','extraction_datetime']


completed_tasks_df = raw_completed_tasks_df[['id','completed_at','user_id','extraction_datetime']]
completed_tasks_df.columns = ['id','completed_at','user_id','extraction_datetime']


opened_tasks_df = raw_opened_tasks_df[[
                        'id','assignee_id','comment_count','created_at',
                        'creator_id','description','due','labels','order',
                        'parent_id','priority','url','extraction_datetime'
                        ]]

opened_tasks_df.columns =[
                        'id','assignee_id','comment_count','created_at',
                        'creator_id','description','due','labels','order',
                        'parent_id','priority','url','extraction_datetime']


opened_tasks_df['due'] = opened_tasks_df['due'].apply(lambda x: datetime.strptime(x['date'], '%Y-%m-%d') if x is not None else None)


tasks_df

/tmp/ipykernel_4261/183686452.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opened_tasks_df['due'] = opened_tasks_df['due'].apply(lambda x: datetime.strptime(x['date'], '%Y-%m-%d') if x is not None else None)


,id,project_id,section_id,content,extraction_datetime
0,6202273884,2298360468,None,Whatch the video: How to use toDoist,2023-12-25 06:48:10
1,6913897305,2298360468,None,Tirar o lixo,2023-12-25 06:48:10
2,6917637603,2298360468,None,Despertar + meditar na palavra + tomar café,2023-12-25 06:48:10
3,7024838788,2315713112,127987968,Encontrar um bom banco de dados de demanda ene...,2023-12-25 06:48:10
4,7024838794,2315713112,127987968,Analisar cases envolvendo análise de series te...,2023-12-25 06:48:10
...,...,...,...,...,...
444,6927120270,2299428438,122944882,Construindo o portifólio,2023-12-30 14:06:33
445,6927120252,2299428438,122944882,Construindo o portifólio,2023-12-30 14:06:33
446,6927120231,2299428438,122944882,Construindo o portifólio,2023-12-30 14:06:33
447,6927120212,2299428438,122944882,Construindo o portifólio,2023-12-30 14:06:33


In [58]:
from Utils.strtime_handler import *

In [4]:
from DatabaseHandler.PostgresHandler import PostgresHandler
db = PostgresHandler()
db.connect('todoist')
db.read_data('SELECT COUNT(*) FROM openedtasks;')['count'][0]

numpy.int64

In [3]:
from ETL.todoist.TodoistMetricCalculator import TodoistMetricCalculator

metric_calculator = TodoistMetricCalculator()

metric_calculator.get_comments_by_word('GPT')

['I learned the following points:\n\n1- Simpler prompt,s no context, can reach better results than complex ones in some contexts. In case of translator, context can add unecessary information. So first focus on clear prompts\n\n2 - GPT has some bias: it tends to add extra columns and misuse key words. Subsequently stages with prompts should be used to mitigate these bias.\n\n3- Ensure consistent output, because LLM are non-deterministc models. The more output you generate more precise it is. In the final of account we enjoy the advantage of variablity to select the most relevant output.']